# "Bin" statistics & histogram

Statistics "bin" are counted on the sample of a single continuous variable.
Firstly, it divides the range of values into bins (sequential, non-overlapping sections),
and then it counts the number of observations in each bin.
It's weighted, it means the weighted count for each bin is calculated
(each element within a bin counted along with its weight).
It's really important to carefully choose a bin-constructing method
(for example, by the exact number of bins or by their width).
This decision has a big impact on how the data is shown and studied.
It makes sure that the way the data is shown is natural to understand and gives a true picture of the information.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

Binning is commonly used in statistics and data analysis
to simplify complex data sets and make them easier to interpret.
Histogram (or any other plot with "bin" statistics) helps to give an overview of the sample distribution.

## Arguments

* Input (mandatory):
    - `x` — numeric sample on which the statistics are calculated
* Weights (optional):
    - `weights` — set of weights of the same size as the input sample.
      `null` (by default) means all weights are equal to `1.0` and the weighted count is equal to the normal one
* Parameters (optional):
    - `binsOption: BinsOption` — specifies either the number of bins or their width:
        - `BinsOption.byNumber(n: Int)` — values are divided into `n` bins (bins width is derived)
        - `BinsOption.byWidth(width: Double)` — values are divided into bins of width `width`
          (the number of bins is derived)
    - `binsAlign: BinsAlign` — specifies bins aligning:
        - `BinsAlign.center(pos: Double)` — bins are aligned by centering bin in `pos`
        - `BinsAlign.boundary(pos: Double)` — bins are aligned by boundary between two bins in `pos`
        - `BinsAlign.none()` — no aligning

### Generalized signature

The specific signature depends on the function,
but all functions related to "bin" statistic
(which will be discussed further below - different variations of `statBin()`, `histogram()`)
have approximately the same signature with the arguments above:

```
statBinArgs := 
   x, 
   weights = null, 
   binsOption: BinsOption = BinsOption.byNumber(20), 
   binsAlign: BinsAlign = BinsAlign.center(0.0)
```

The possible types of `x` and `weights` depend on where a certain function is used.
They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame`
(`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name                 | type   | description                                              |
|----------------------|--------|----------------------------------------------------------|
| Stat.x               | Double | Center of bin                                            |
| Stat.count           | Int    | Number of observations in this bin                       |
| Stat.countWeighted   | Double | Weighted count (sum of observations weights in this bin) |
| Stat.density         | Double | Empirically estimated density in this bin                |
| Stat.densityWeighted | Double | Weighted density                                         |

## StatBin plots

In [1]:
%use kandy
%use dataframe

import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import org.jetbrains.kotlinx.statistics.dataframe.stat.mean

In [2]:
// Generate sample from normal distribution
val depthList = NormalDistribution(500.0, 100.0).sample(1000).toList()
// Generate sample from uniform distribution
val coeffList = UniformRealDistribution(0.0, 1.0).sample(1000).toList()
// gather them into the DataFrame
val df = dataFrameOf(
    "depth" to depthList,
    "coeff" to coeffList
)
df.head()

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-117440512"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

`df` has a signature

| depth | coeff |
|-------|-------|

Let's take a look at `StatBin` output DataFrame:

In [3]:
df.statBin("depth", "coeff", binsOption = BinsOption.byNumber(10))

<iframe onload="o_resize_iframe_out_2()" style="width:100%;" class="result_container" id="iframe_out_2" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-117440511"></table>

<p class="dataframe_description">DataFrame: rowsCount = 10, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for

It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>count</th>
      <th>countWeighted</th>
      <th>density</th>
      <th>densityWeighted</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics.
For `statBin`, each row corresponds to one bin.
`Stat.x` is the column with the centers of the bins.
`Stat.count` contains the number of observations in the bin.
`Stat.countWeighted` — weighted version of `count`.
There are also `Stat.density` and `Stat.densityWeighted`.
They contain empirically estimated density (both normal and weighted)
of the sample in the points corresponding to the centers of bins.

`DataFrame` with "bin" statistics is called `StatBinFrame`

### `statBin` context transform

`statBin(statBinArgs) { /*new plotting context*/ }` modifies a plotting context — instead of original data
(no matter was it empty or not) new `StatBin` dataset (calculated on given arguments.
Inputs and weights can be provided as `Iterable` or as dataset column reference — by name as a `String`,
as a `ColumnReference` or as a `DataColumn`) is used inside a new context
(original dataset and primary context are not affected —
you can add layers using initial dataset outside the `statBin` context).
Since the old dataset is irrelevant, we cannot use references for its columns.
But we can refer to the new ones.
They are all contained in the `Stat` group and can be called inside the new context:

In [4]:
plot {
    statBin(depthList, binsAlign = BinsAlign.center(500.0)) {
        // new `StatBin` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.count)
            fillColor = Color.RED
            alpha = 0.5
        }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3Bq1hH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[221.3233170364405,252.28739292128043,283.25146880612033,314.2155446909603,345.1796205758003,376.1436964606402,407.10777234548016,438.0718482303201,469.03592411516,500.0,530.96407588484,561.9281517696797,592.8922276545197,623.8563035393597,654.8203794241997,685.7844553090397,716.7485311938794,747.7126070787194,778.6766829635594,809.6407588483994],
"count":[3.0,7.0,12.0,26.0,37.0,48.0,79.0,115.0,125.0,128.0,112.0,99.0,76.0,53.0,40.0,21.0,10.0,4.0,4.0,1.0]
},
"sampling":"none",
"alpha":0.5,
"position":"identity",
"geom":"area",
"fill":"#ee6666"
}]
};
 var plotContainer = document.getElementById("3Bq1hH");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.411735619708907 336.32142857142856 L24.411735619708907 336.32142857142856 L50.10829942992876 326.0833333333333 L75.80486324014859 313.2857142857143 L101.50142705036853 277.45238095238096 L127.19799086058845 249.29761904761904 L152.89455467080825 221.14285714285717 L178.59111848102816 141.79761904761907 L204.28768229124802 49.654761904761926 L229.98424610146787 24.059523809523853 L255.6808099116878 16.380952380952408 L281.3773737219077 57.33333333333337 L307.07393753212745 90.60714285714289 L332.77050134234736 149.47619047619048 L358.4670651525672 208.3452380952381 L384.16362896278713 241.61904761904762 L409.86019277300704 290.25 L435.55675658322684 318.4047619047619 L461.25332039344676 333.76190476190476 L486.94988420366667 333.76190476190476 L512.6464480138866 341.4404761904762 L512.6464480138866 344.0 L486.94988420366667 344.0 L461.25332039344676 344.0 L435.55675658322684 344.0 L409.86019277300704 344.0 L384.16362896278713 344.0 L358.4670651525672 344.0 L332.77050134234736 344.0 L307.07393753212745 344.0 L281.3773737219077 344.0 L255.6808099116878 344.0 L229.98424610146787 344.0 L204.28768229124802 344.0 L178.59111848102816 344.0 L152.89455467080825 344.0 L127.19799086058845 344.0 L101.50142705036853 344.0 L75.80486324014859 344.0 L50.10829942992876 344.0 L24.411735619708907 344.0 Z" fill="rgb(238,102,102)" stroke-width="1.0" fill-opacity="0.5019607843137255">
 
 
 
 <path d="M24.411735619708907 336.32142857142856 L24.411735619708907 336.32142857142856 L50.10829942992876 326.0833333333333 L75.80486324014859 313.2857142857143 L101.50142705036853 277.45238095238096 L127.19799086058845 249.29761904761904 L152.89455467080825 221.14285714285717 L178.59111848102816 141.79761904761907 L204.28768229124802 49.654761904761926 L229.98424610146787 24.059523809523853 L255.6808099116878 16.380952380952408 L281.3773737219077 57.33333333333337 L307.07393753212745 90.60714285714289 L332.77050134234736 149.47619047619048 L358.4670651525672 208.3452380952381 L384.16362896278713 241.61904761904762 L409.86019277300704 290.25 L435.55675658322684 318.4047619047619 L461.25332039344676 333.76190476190476 L486.94988420366667 333.76190476190476 L512.6464480138866 341.4404761904762 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 
 count
 
 
 
 
 x

### Histogram layer

A histogram is a statistical chart that serves to visually approximate the distribution of a numerical variable.
It's a bar plot where each bar is representing a bin: its x coordinate is corresponding to bin range and y to count.
So basically, we can build a histogram with `statBin` as follows:

In [5]:
val statBinBarsPlot = df.plot {
    statBin("depth") {
        bars {
            x(Stat.x)
            y(Stat.count)
        }
    }
    layout.title = "`statBin` + `bars`"
}
statBinBarsPlot

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="R3BWfJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"`statBin` + `bars`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[216.7485311938796,247.71260707871951,278.6766829635594,309.6407588483994,340.6048347332394,371.56891061807926,402.53298650291924,433.4970623877592,464.4611382725991,495.4252141574391,526.389290042279,557.3533659271188,588.3174418119588,619.2815176967988,650.2455935816388,681.2096694664788,712.1737453513185,743.1378212361585,774.1018971209985,805.0659730058385],
"count":[3.0,4.0,11.0,25.0,39.0,45.0,71.0,104.0,134.0,127.0,110.0,103.0,82.0,52.0,47.0,23.0,10.0,5.0,4.0,1.0]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("R3BWfJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 140
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `statBin` + `bars`
 
 
 
 
 count
 
 
 
 
 x

But we can do it even faster with `histogram(statBinArgs)` method:

In [6]:
val histogramPlot = plot {
    histogram(depthList)
    layout.title = "`histogram`"
}
histogramPlot

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="syDZUb"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"`histogram`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[216.7485311938796,247.71260707871951,278.6766829635594,309.6407588483994,340.6048347332394,371.56891061807926,402.53298650291924,433.4970623877592,464.4611382725991,495.4252141574391,526.389290042279,557.3533659271188,588.3174418119588,619.2815176967988,650.2455935816388,681.2096694664788,712.1737453513185,743.1378212361585,774.1018971209985,805.0659730058385],
"count":[3.0,4.0,11.0,25.0,39.0,45.0,71.0,104.0,134.0,127.0,110.0,103.0,82.0,52.0,47.0,23.0,10.0,5.0,4.0,1.0]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("syDZUb");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 140
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `histogram`
 
 
 
 
 count
 
 
 
 
 x

Let's compare them:

In [7]:
plotGrid(listOf(statBinBarsPlot, histogramPlot))

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="TdyfKd"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"`statBin` + `bars`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[3.0,4.0,11.0,25.0,39.0,45.0,71.0,104.0,134.0,127.0,110.0,103.0,82.0,52.0,47.0,23.0,10.0,5.0,4.0,1.0],
"x":[216.7485311938796,247.71260707871951,278.6766829635594,309.6407588483994,340.6048347332394,371.56891061807926,402.53298650291924,433.4970623877592,464.4611382725991,495.4252141574391,526.389290042279,557.3533659271188,588.3174418119588,619.2815176967988,650.2455935816388,681.2096694664788,712.1737453513185,743.1378212361585,774.1018971209985,805.0659730058385]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
},{
"ggtitle":{
"text":"`histogram`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[3.0,4.0,11.0,25.0,39.0,45.0,71.0,104.0,134.0,127.0,110.0,103.0,82.0,52.0,47.0,23.0,10.0,5.0,4.0,1.0],
"x":[216.7485311938796,247.71260707871951,278.6766829635594,309.6407588483994,340.6048347332394,371.56891061807926,402.53298650291924,433.4970623877592,464.4611382725991,495.4252141574391,526.389290042279,557.3533659271188,588.3174418119588,619.2815176967988,650.2455935816388,681.2096694664788,712.1737453513185,743.1378212361585,774.1018971209985,805.0659730058385]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
}],
"kind":"subplots"
};
 var plotContainer = document.getElementById("TdyfKd");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 140
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `statBin` + `bars`
 
 
 
 
 count
 
 
 
 
 x
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 140
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `histogram`
 
 
 
 
 count
 
 
 
 
 x

These two plots are identical.
Indeed, `histogram` just uses `statBin` and `bars` and performs coordinates mappings under the hood.
And we can customize histogram layer: `histogram()` optionally opens a new context,
where we can configure bars (as in the usual context opened by `bars { ... }`) —
even change coordinate mappings from default ones.
`StatBin` dataset of histogram is also can be accessed here.

In [8]:
df.plot {
    histogram(depth, binsAlign = BinsAlign.center(500.0)) {
        // Change a column mapped on `y` to `Stat.density`
        y(Stat.density)
        // Filling color depends on `density` statistic
        fillColor(Stat.density) {
            scale = continuous(Color.YELLOW..Color.RED)
        }
        borderLine.color = Color.BLACK
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="aqPQoF"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#fac858",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"fill":"density"
},
"stat":"identity",
"data":{
"density":[9.688646969983706E-5,2.2606842929961978E-4,3.875458787993482E-4,8.396827373985877E-4,0.0011949331262979903,0.001550183515197393,0.002551343702095709,0.0037139813384937537,0.0040369362374932105,0.004133822707193048,0.0036170948687939165,0.003197253500094623,0.002454457232395872,0.0017116609646971213,0.0012918195959978273,6.782052878988595E-4,3.229548989994568E-4,1.2918195959978274E-4,1.2918195959978274E-4,3.2295489899945685E-5],
"x":[221.3233170364405,252.28739292128043,283.25146880612033,314.2155446909603,345.1796205758003,376.1436964606402,407.10777234548016,438.0718482303201,469.03592411516,500.0,530.96407588484,561.9281517696797,592.8922276545197,623.8563035393597,654.8203794241997,685.7844553090397,716.7485311938794,747.7126070787194,778.6766829635594,809.6407588483994]
},
"color":"#000000",
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("aqPQoF");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000
 
 
 
 
 
 
 0.0005
 
 
 
 
 
 
 0.0010
 
 
 
 
 
 
 0.0015
 
 
 
 
 
 
 0.0020
 
 
 
 
 
 
 0.0025
 
 
 
 
 
 
 0.0030
 
 
 
 
 
 
 0.0035
 
 
 
 
 
 
 0.0040
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 density
 
 
 
 
 depth
 
 
 
 
 
 
 
 
 density
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.001
 
 
 
 
 
 
 
 
 0.002
 
 
 
 
 
 
 
 
 0.003
 
 
 
 
 
 
 
 
 0.004

If we specify weights, `Stat.countWeighted` is mapped to `y` by default:

In [9]:
df.plot {
    // Count sample mean
    val mean = depth.mean()
    // Add weighted histogram
    histogram(depth, coeff, binsOption = BinsOption.byNumber(10), binsAlign = BinsAlign.boundary(mean))
    // We can add other layers as well.
    // Let's add a vertical mark line in the mean of sample
    vLine {
        xIntercept.constant(mean)
        tooltips { line("Depth mean: ${String.format("%.2f", mean)}m") }
        color = Color.RED; width = 3.0
    }
    x.axis.name = "depth, m"
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cyfE77"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"depth, m",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"countWeighted"
},
"stat":"identity",
"data":{
"x":[219.29538727761064,281.22353904729056,343.1516908169704,405.0798425866503,467.0079943563302,528.93614612601,590.86429789569,652.7924496653698,714.7206014350497,776.6487532047296],
"countWeighted":[2.882473082614795,14.882791239223723,36.89365886700907,75.31912991408868,121.1226349299481,113.7199446295252,69.30883702451375,38.79467285601214,14.421768296171138,4.1742802133835575]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
},{
"mapping":{
},
"stat":"identity",
"color":"#ee6666",
"size":3.0,
"sampling":"none",
"position":"identity",
"xintercept":497.97207024117006,
"geom":"vline",
"tooltips":{
"lines":["Depth mean: 497,97m"]
},
"data":{
}
}]
};
 var plotContainer = document.getElementById("cyfE77");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 countWeighted
 
 
 
 
 depth, m

### `histogram` plot

`histogram(statBinArgs)` and `DataFrame.histogram(statBinArgs)` are a family of functions for fast plotting a histogram.

In [10]:
histogram(depthList, binsAlign = BinsAlign.center(500.0))

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ETnegx"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[221.3233170364405,252.28739292128043,283.25146880612033,314.2155446909603,345.1796205758003,376.1436964606402,407.10777234548016,438.0718482303201,469.03592411516,500.0,530.96407588484,561.9281517696797,592.8922276545197,623.8563035393597,654.8203794241997,685.7844553090397,716.7485311938794,747.7126070787194,778.6766829635594,809.6407588483994],
"count":[3.0,7.0,12.0,26.0,37.0,48.0,79.0,115.0,125.0,128.0,112.0,99.0,76.0,53.0,40.0,21.0,10.0,4.0,4.0,1.0]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("ETnegx");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 x

In [11]:
df.histogram("depth")

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Tz3pTy"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[216.7485311938796,247.71260707871951,278.6766829635594,309.6407588483994,340.6048347332394,371.56891061807926,402.53298650291924,433.4970623877592,464.4611382725991,495.4252141574391,526.389290042279,557.3533659271188,588.3174418119588,619.2815176967988,650.2455935816388,681.2096694664788,712.1737453513185,743.1378212361585,774.1018971209985,805.0659730058385],
"count":[3.0,4.0,11.0,25.0,39.0,45.0,71.0,104.0,134.0,127.0,110.0,103.0,82.0,52.0,47.0,23.0,10.0,5.0,4.0,1.0]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("Tz3pTy");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 140
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 depth

In case you want to provide inputs and weights using column selection DSL, it's a bit different from the usual one —
you should assign `x` input and (optionally) `weight` throw invocation eponymous functions:

In [12]:
df.histogram(binsOption = BinsOption.byNumber(10)) {
    x(depth)
    weight(coeff)
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="zD7zFF"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"countWeighted"
},
"stat":"identity",
"data":{
"x":[216.74853119387961,278.6766829635595,340.60483473323933,402.53298650291924,464.4611382725991,526.3892900422791,588.3174418119588,650.2455935816388,712.1737453513186,774.1018971209985],
"countWeighted":[2.882473082614795,13.891579461215857,33.627782938549736,73.64342482732106,122.07315440232745,115.28963425731064,72.13662991779455,38.36929432008623,15.431937631886333,4.1742802133835575]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("zD7zFF");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 countWeighted
 
 
 
 
 depth

Histogram plot can be configured with `.configure {}` extension —
it opens a context that combines bars, `StatBin` and plot context.
That means you can configure bars settings, mappings using `StatBin` dataset and any plot adjustments:

In [13]:
df.histogram(binsOption = BinsOption.byNumber(15)) {
    x(depth)
}.configure {
    // Bars + StatBin + PlotContext
    // Can't add a new layer
    x.limits = 100..900
    // Can add bar mapping, include on `Stat.*` columns
    fillColor(Stat.count) { scale = continuous(Color.GREEN..Color.RED) }
    // Can configure general plot adjustments
    layout {
        title = "Configured histogram plot"
        size = 600 to 350
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="XcHodZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Configured histogram plot"
},
"mapping":{
},
"data":{
"x":[227.06988982215955,268.3553243352795,309.6407588483994,350.92619336151927,392.21162787463925,433.4970623877591,474.78249690087904,516.067931413999,557.3533659271188,598.6388004402388,639.9242349533588,681.2096694664785,722.4951039795985,763.7805384927185,805.0659730058384],
"count":[6.0,8.0,35.0,53.0,87.0,138.0,170.0,156.0,139.0,95.0,63.0,31.0,12.0,6.0,1.0]
},
"ggsize":{
"width":600.0,
"height":350.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[100.0,900.0]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#3ba272",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"depth",
"limits":[100.0,900.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"count"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("XcHodZ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 50
 
 
 
 
 
 
 100
 
 
 
 
 
 
 150
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Configured histogram plot
 
 
 
 
 count
 
 
 
 
 depth
 
 
 
 
 
 
 
 
 count
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 150

## Grouped `staBin`

`statBin` can be applied for grouped data — statistics will be counted on each group independently but with equal bins.
This application returns a new `GroupBy`
dataset with the same keys as the old one but with `StatBin` groups instead of old ones.

In [14]:
// Create two samples from normal distribution with different mean/std
val rangesA = NormalDistribution(500.0, 100.0).sample(5000).toList()
val rangesB = NormalDistribution(400.0, 80.0).sample(5000).toList()

// Gather them into `DataFrame` with "A" and "B" keys in the "category" column
val rangesDF = dataFrameOf(
    "range" to rangesA + rangesB,
    "category" to List(5000) { "A" } + List(5000) { "B" }
)
rangesDF.head(5)

<iframe onload="o_resize_iframe_out_3()" style="width:100%;" class="result_container" id="iframe_out_3" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-117440510"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

It has the following signature:

| range | category |
|-------|----------|

In [15]:
// Group it by "category"
val groupedRangesDF = rangesDF.groupBy { category }
groupedRangesDF

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-117440509"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[range|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [16]:
groupedRangesDF.statBin { x(range) }

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-117440506"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++

After `statBin` applying it's still a `GroupBy` but with different signature of `group` —
all groups have the same signature as usual `DataFrame` after `statBin` applying (i.e. `StatBinFrame`):

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: StaBinFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statBin` transformation within groups, the grouping keys did not change.
Also, all bin centers match — it helps to build grouped histogram.

The plotting process doesn't change much — we do everything the same.

In [17]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
        }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oNCaH4"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"density":[1.0937128673171504E-5,2.1874257346343008E-5,7.109133637561477E-5,1.6405693009757254E-4,3.8279950356100265E-4,6.999762350829762E-4,0.001367141084146438,0.002313202714375773,0.003226452958585593,0.0034342584033758517,0.0038498692929563694,0.00378424652091734,0.0031608301865465645,0.002067117319229414,0.0013835467771561951,7.05444799419562E-4,4.5935940427320313E-4,1.3124554407805802E-4,8.749702938537203E-5,2.1874257346343008E-5,2.2186266012526162E-5,9.614048605428003E-5,3.9195736622129555E-4,9.096369065135728E-4,0.0021446723812108626,0.0034980346079749587,0.005568752769144068,0.006604111849728622,0.0062195499055115,0.005383867219039683,0.003409289543924854,0.0017009470609603392,7.765193104384158E-4,1.848855501043847E-4,6.655879803757849E-5,0.0,0.0,0.0,0.0,0.0],
"x":[129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869,129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869]
},
"sampling":"none",
"position":"identity",
"geom":"area"
}]
};
 var plotContainer = document.getElementById("oNCaH4");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000
 
 
 
 
 
 
 0.001
 
 
 
 
 
 
 0.002
 
 
 
 
 
 
 0.003
 
 
 
 
 
 
 0.004
 
 
 
 
 
 
 0.005
 
 
 
 
 
 
 0.006
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.8183676303992 342.89937609982405 L23.8183676303992 342.89937609982405 L48.89033355713517 339.2306297659041 L73.96229948387119 324.5556444302245 L99.03426541060719 298.87442009278516 L124.10623133734316 237.60635631632272 L149.1781972640792 170.46829840558846 L174.25016319081516 67.74340105583104 L199.32212911755113 16.380952380952408 L224.39409504428716 35.45843331733596 L249.46606097102313 76.91526689063085 L274.53802689775915 174.87079400629233 L299.6099928244951 259.61883431984216 L324.68195875123115 305.478163493841 L349.7539246779671 334.82813416520025 L374.82589060470315 340.6981282994721 L399.8978565314392 344.0 L424.9698224581751 344.0 L450.0417883849111 344.0 L475.1137543116471 344.0 L500.18572023838306 344.0 L500.18572023838306 344.0 L475.1137543116471 344.0 L450.0417883849111 344.0 L424.9698224581751 344.0 L399.8978565314392 344.0 L374.82589060470315 344.0 L349.7539246779671 344.0 L324.68195875123115 344.0 L299.6099928244951 344.0 L274.53802689775915 344.0 L249.46606097102313 344.0 L224.39409504428716 344.0 L199.32212911755113 344.0 L174.25016319

As you can see, there are two areas because we have two groups of data.
To distinguish them, we need to add mapping to the filling color from the key.
This is convenient — the key is available in the context

In [18]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
            // can access "key." columns and create mapping from them
            fillColor(key.category)
            alpha = 0.6
        }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Og5oJR"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"density":[1.0937128673171504E-5,2.1874257346343008E-5,7.109133637561477E-5,1.6405693009757254E-4,3.8279950356100265E-4,6.999762350829762E-4,0.001367141084146438,0.002313202714375773,0.003226452958585593,0.0034342584033758517,0.0038498692929563694,0.00378424652091734,0.0031608301865465645,0.002067117319229414,0.0013835467771561951,7.05444799419562E-4,4.5935940427320313E-4,1.3124554407805802E-4,8.749702938537203E-5,2.1874257346343008E-5,2.2186266012526162E-5,9.614048605428003E-5,3.9195736622129555E-4,9.096369065135728E-4,0.0021446723812108626,0.0034980346079749587,0.005568752769144068,0.006604111849728622,0.0062195499055115,0.005383867219039683,0.003409289543924854,0.0017009470609603392,7.765193104384158E-4,1.848855501043847E-4,6.655879803757849E-5,0.0,0.0,0.0,0.0,0.0],
"x":[129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869,129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"area"
}]
};
 var plotContainer = document.getElementById("Og5oJR");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000
 
 
 
 
 
 
 0.001
 
 
 
 
 
 
 0.002
 
 
 
 
 
 
 0.003
 
 
 
 
 
 
 0.004
 
 
 
 
 
 
 0.005
 
 
 
 
 
 
 0.006
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M19.865713083991892 342.89937609982405 L19.865713083991892 342.89937609982405 L40.776990014509664 339.2306297659041 L61.688266945027436 324.5556444302245 L82.59954387554524 298.87442009278516 L103.51082080606298 237.60635631632272 L124.42209773658078 170.46829840558846 L145.33337466709855 67.74340105583104 L166.24465159761633 16.380952380952408 L187.15592852813413 35.45843331733596 L208.0672054586519 76.91526689063085 L228.97848238916967 174.87079400629233 L249.88975931968744 259.61883431984216 L270.80103625020524 305.478163493841 L291.71231318072296 334.82813416520025 L312.6235901112408 340.6981282994721 L333.5348670417586 344.0 L354.44614397227633 344.0 L375.35742090279416 344.0 L396.2686978333119 344.0 L417.1799747638296 344.0 L417.1799747638296 344.0 L396.2686978333119 344.0 L375.35742090279416 344.0 L354.4461439722

The `histogram` layer also works.
Moreover, if we have exactly one grouping key, a mapping from it to `fillColor` will be created by default.

In [19]:
groupedRangesDF.plot {
    histogram(range)
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="dmr5Er"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869,129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869],
"count":[2.0,4.0,13.0,30.0,70.0,128.0,250.0,423.0,590.0,628.0,704.0,692.0,578.0,378.0,253.0,129.0,84.0,24.0,16.0,4.0,3.0,13.0,53.0,123.0,290.0,473.0,753.0,893.0,841.0,728.0,461.0,230.0,105.0,25.0,9.0,0.0,0.0,0.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("dmr5Er");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

We can customize it like we used to.
From the differences - access to `key` columns, and we can customize the `position`
of bars (within a single x-coordinate), for example — stack them:

In [20]:
groupedRangesDF.plot {
    histogram(range) {
        fillColor(key.category) {
            scale = categorical(listOf(Color.GREEN, Color.ORANGE))
        }
        borderLine.width = 0.0
        width = 1.0
        // Adjust position of bars from different groups
        position = Position.stack()
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ANMfUb"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#3ba272","#fc8452"]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"count":[2.0,4.0,13.0,30.0,70.0,128.0,250.0,423.0,590.0,628.0,704.0,692.0,578.0,378.0,253.0,129.0,84.0,24.0,16.0,4.0,3.0,13.0,53.0,123.0,290.0,473.0,753.0,893.0,841.0,728.0,461.0,230.0,105.0,25.0,9.0,0.0,0.0,0.0,0.0,0.0],
"x":[129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869,129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"size":0.0,
"sampling":"none",
"width":1.0,
"position":"stack",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("ANMfUb");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 200
 
 
 
 
 
 
 400
 
 
 
 
 
 
 600
 
 
 
 
 
 
 800
 
 
 
 
 
 
 1,000
 
 
 
 
 
 
 1,200
 
 
 
 
 
 
 1,400
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

Histogram plot for `GroupBy` (i.e. `GroupBy.histogram(statBinArgs)` extensions) works as well:

In [21]:
groupedRangesDF.histogram("range")

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7VZvmK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869,129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869],
"count":[2.0,4.0,13.0,30.0,70.0,128.0,250.0,423.0,590.0,628.0,704.0,692.0,578.0,378.0,253.0,129.0,84.0,24.0,16.0,4.0,3.0,13.0,53.0,123.0,290.0,473.0,753.0,893.0,841.0,728.0,461.0,230.0,105.0,25.0,9.0,0.0,0.0,0.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("7VZvmK");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

... and can be configured the same way:

In [22]:
groupedRangesDF.histogram(binsAlign = BinsAlign.center(500.0)) { x(range) }.configure {
    alpha = 0.6
    // make the bars from different groups overlap with each other
    position = Position.identity()
    // can access key column by name as `String`
    fillColor("category") { scale = categoricalColorBrewer(BrewerPalette.Qualitative.Dark2) }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EXMnp7"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[131.0916588929391,167.98249300364517,204.87332711435127,241.76416122505734,278.65499533576343,315.5458294464695,352.4366635571756,389.32749766788174,426.21833177858775,463.1091658892939,500.0,536.890834110706,573.7816682214121,610.6725023321183,647.5633364428244,684.4541705535304,721.3450046642365,758.2358387749426,795.1266728856486,832.0175069963548,131.0916588929391,167.98249300364517,204.87332711435127,241.76416122505734,278.65499533576343,315.5458294464695,352.4366635571756,389.32749766788174,426.21833177858775,463.1091658892939,500.0,536.890834110706,573.7816682214121,610.6725023321183,647.5633364428244,684.4541705535304,721.3450046642365,758.2358387749426,795.1266728856486,832.0175069963548],
"count":[3.0,3.0,14.0,30.0,71.0,132.0,258.0,426.0,599.0,639.0,708.0,687.0,561.0,372.0,252.0,123.0,81.0,22.0,15.0,4.0,3.0,14.0,55.0,129.0,295.0,497.0,757.0,900.0,830.0,715.0,453.0,225.0,96.0,24.0,7.0,0.0,0.0,0.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_brewer",
"palette":"Dark2"
},{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("EXMnp7");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [23]:
rangesDF.plot {
    groupBy(category) {
        histogram(range)
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="h70MNj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869,129.11791938747132,166.00875349817738,202.89958760888348,239.79042171958957,276.68125583029564,313.57208994100176,350.46292405170783,387.3537581624139,424.24459227312,461.1354263838261,498.0262604945322,534.9170946052383,571.8079287159444,608.6987628266504,645.5895969373565,682.4804310480627,719.3712651587687,756.2620992694748,793.1529333801809,830.0437674908869],
"count":[2.0,4.0,13.0,30.0,70.0,128.0,250.0,423.0,590.0,628.0,704.0,692.0,578.0,378.0,253.0,129.0,84.0,24.0,16.0,4.0,3.0,13.0,53.0,123.0,290.0,473.0,753.0,893.0,841.0,728.0,461.0,230.0,105.0,25.0,9.0,0.0,0.0,0.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("h70MNj");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B